In [1]:
%matplotlib inline
from random import *
import csv
import CommonVideo
import model
import StuDrop

In [8]:
# Video Infro
with open('VideoData_Level.csv') as tmp_file:
    video_data = list(csv.DictReader(tmp_file)) 
    
# Selecting Proper amount of student
all_StuList = []
for key in CommonVideo.StuVideoDict:
    all_StuList.append(key)

# Training Data
stuList = all_StuList[0:2400]
stu_clusters = model.clustering_model(stuList, vis = False)


# of clusters=> 738


In [15]:
# Testing Data

Testing = all_StuList[2400:]
Testing_Count = len(Testing)
TP = 0
FP = 0
TN = 0
FN = 0
for student in Testing :
    # Random Cut
    Target = student
    #rand = randint(1,9)/10
    rand = 0.6
    SplitNumber,Now,NowPer,Next= StuDrop.SplitSection(student,rand)   #(拆了多少人 , 上一位 , 目前比例 , 下一位 )
    #print(rand, SplitNumber ,Next)
    # Cluster
    belong = model.find_closerclusering(Target, stu_clusters , stuList)
    TargetList = list(set(CommonVideo.video_order[Target][:SplitNumber]))
    
    # Prediction for continue
    Score = model.Prediction(TargetList , belong , stu_clusters , stuList ,video_data)  #預測看完的分段 
    for (VideoId , TruthRate) in StuDrop.DropList(Target):       #實際情形
        if VideoId == CommonVideo.video_order[Target][SplitNumber] :
            Truth = TruthRate
    # Score >= Truth : Negative
        #  Score >= Now : T
        #  Score < Now : F
    # Score < Truth : Postive
        #  Score >= Now : T
        #  Score < Now : F

    if Score >= Truth and Score >= NowPer:
        TN = TN + 1
    elif Score >= Truth and Score < NowPer:
        FN = FN + 1
    elif Score < Truth and Score >= NowPer:
        TP = TP + 1
    elif Score < Truth and Score < NowPer:
        FP = FP + 1
    

In [16]:
#Recall
Recall = TP / (TP + FN) 
#Percision
Precision = TP / (TP + FP)
# F1
F1 = 2*Precision*Recall / (Recall + Precision)




print(Recall , Precision , F1)

1.0 0.49927849927849927 0.6660250240615977


In [17]:
print(TP,
FP ,
TN ,
FN )

346 347 3 0
